## Loading modules & data

In [ ]:
#! pip install mne

In [ ]:
# importing modules
import numpy as np
import mne
import helper_functions as hf
import matplotlib.pyplot as plt
from mne.time_frequency import tfr_morlet
from mne.stats import spatio_temporal_cluster_test, combine_adjacency
from mne.channels import find_ch_adjacency

# Time-frequency analysis

#### Reading in epochs
The epochs saved in the `ICA.ipynb` is loaded in for analysis. The data has been cleaned using ICA, time-locked to the reaction time and downsampled to 250 Hz.

In [ ]:
epochs = mne.read_epochs('Stroop_mouse_EEG_data/epochs/epochs_RT_epo.fif')

In [ ]:
leftcentralpicks = [5, 6, 10]
mne.viz.plot_epochs_image(epochs, group_by={'Left central picks' : leftcentralpicks}, combine = 'mean', cmap = 'interactive', vmin=-40, vmax=40)

Now lets look into what causes these deflections. This is done by making a time-frequency analysis of the epochs. 

#### Time-Frequency analysis

In [ ]:
#freqs = np.logspace(*np.log10([6, 35]), num=12)
freqs = np.arange(7, 30, 1)
n_cycles = freqs / 2.  # different number of cycle per frequency
power, itc =  mne.time_frequency.tfr_morlet(epochs, freqs=freqs, n_cycles=n_cycles, use_fft=True,
                        return_itc=True, decim=3, n_jobs=1)

#### Plotting Alpha waves

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(12, 6), dpi = 300)

baseline = (0.3, 0.5)
power.plot_topomap(ch_type='eeg', tmin=-0.5, tmax=-0.25, fmin=7, fmax=12,
                   baseline=baseline, mode='logratio', axes=axis[0],
                   title='Alpha (-0.5 to -0.25)', show=False, vmin = -.16, vmax = .16)

power.plot_topomap(ch_type='eeg', tmin=-0.25, tmax=0.1, fmin=7, fmax=12,
                   baseline=baseline, mode='logratio', axes=axis[1],
                   title='Alpha (-0.25 to 0.1)', show=False, vmin = -.16, vmax = .16)

mne.viz.tight_layout()
plt.savefig('figures/time_frequency_alpha_topomaps.png')
plt.show()

In [ ]:
# NOTE:See if we can take some kind of average to include all channels in one plot???
power.plot(mode='mean', picks = leftcentralpicks,baseline = (0.3, 0.5), tmin = -.7, tmax = .6, fmin = 7, fmax = 12);

We see that the alpha waves are reduced just before and while the action is done and then rebounds afterwards. alpha waves "locks" a given network in a dominating rhythm, and when this rhythm is supressed it opens up for other processing in the given area (i.e, initiating a motor action).

#### Plotting beta rhythms

In [ ]:
power.plot(mode='mean', picks = leftcentralpicks,baseline = (0.3, 0.5), tmin = -.7, tmax = .6, fmin = 12, fmax = 30);

#### Contrasting across conditions
How to understand this? higher inhibition of motor reponse??? Super interesting if that is the case.


**Link to code:**
https://berdakh.github.io/blog/eeg/jupyter/2020/09/14/MNE-Tutorial-part-2.html#Time-Frequency-stuff

In [ ]:
n_cycles = freqs / 2.  # different number of cycle per frequency
power_inc, itc_inc =  mne.time_frequency.tfr_morlet(epochs['cInc'], freqs=freqs, n_cycles=n_cycles, use_fft=True,
                        return_itc=True, n_jobs=1)
power_con, itc_con =  mne.time_frequency.tfr_morlet(epochs['cCon'], freqs=freqs, n_cycles=n_cycles, use_fft=True,
                        return_itc=True, n_jobs=1)
power_neu, itc_neu =  mne.time_frequency.tfr_morlet(epochs['cNeu'], freqs=freqs, n_cycles=n_cycles, use_fft=True,
                        return_itc=True, n_jobs=1)
power_all, itc_all =  mne.time_frequency.tfr_morlet(epochs, freqs=freqs, n_cycles=n_cycles, use_fft=True,
                        return_itc=True, n_jobs=1)

## Comparison of mean power in the conditions

**Mu rhythms**
7-12 Hz

**Lower beta rhythms**
12-18 Hz

**Higher beta rhytms**
18-30 Hz

In [ ]:
freq_ind_b = 0
freq_ind_a = 5
time_ind_b = 70
time_ind_a = 300


## averaging over alpha frequencies
average_con = hf.average_power_time(power_con, leftcentralpicks, freq_ind_b, freq_ind_a, time_ind_b, time_ind_a)
average_inc = hf.average_power_time(power_inc, leftcentralpicks, freq_ind_b, freq_ind_a, time_ind_b, time_ind_a)
average_neu = hf.average_power_time(power_neu, leftcentralpicks, freq_ind_b, freq_ind_a, time_ind_b, time_ind_a)


freq_ind_b = 5
freq_ind_a = 11

## averaging over low beta frequencies
average_con_low = hf.average_power_time(power_con, leftcentralpicks, freq_ind_b, freq_ind_a, time_ind_b, time_ind_a)
average_inc_low = hf.average_power_time(power_inc, leftcentralpicks, freq_ind_b, freq_ind_a, time_ind_b, time_ind_a)
average_neu_low = hf.average_power_time(power_neu, leftcentralpicks, freq_ind_b, freq_ind_a, time_ind_b, time_ind_a)

freq_ind_b = 11
freq_ind_a = 22

## averaging over high beta frequencies
#average_con_high = hf.average_power_time(power_con, leftcentralpicks, freq_ind_b, freq_ind_a, time_ind_b, time_ind_a)
#average_inc_high = hf.average_power_time(power_inc, leftcentralpicks, freq_ind_b, freq_ind_a, time_ind_b, time_ind_a)
#average_neu_high = hf.average_power_time(power_neu, leftcentralpicks, freq_ind_b, freq_ind_a, time_ind_b, time_ind_a)

In [ ]:
times = power_con.times[time_ind_b:time_ind_a]

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(14, 6), dpi=300)
axis[0].plot(times, average_con, 'lightblue', label = 'Congruent')
axis[0].plot(times, average_inc, 'steelblue', label = 'Incongruent')
axis[0].axvline(x=0.0, color='black', linestyle='--')
axis[0].title.set_text('7-12 Hz')

axis[1].plot(times, average_con_low, 'lightblue', label = 'Congruent')
axis[1].plot(times, average_inc_low, 'steelblue', label = 'Incongruent')
axis[1].title.set_text('12-18 Hz')
axis[1].axvline(x=0.0, color='black', linestyle='--')
axis[1].legend(loc = 'upper right')

#axis[2].plot(times, average_con_high, 'steelblue', label = 'Congruent')
#axis[2].plot(times, average_inc_high, 'lightblue', label = 'Incongruent')
#axis[2].legend(loc = 'upper right')
#axis[2].title.set_text('18-30 Hz')

fig.suptitle('Average Power (C3, FC1, FC3)', fontsize=18)
fig.savefig('figures/average_power_time.png')

## Cluster-based permutation test on tfr

https://mne.tools/stable/generated/mne.stats.permutation_cluster_test.html

**Notes:** The null hypothesis of this test is that the time frequency representation in the experimental conditions arise from the same probability distribution. Multiple factors influence this type of test, like the signal-to-noise ratio, the threshold chosen to select samples to belong to a cluster and the number of trials. 

In [ ]:
baseline = (0.3, 0.5)

**Note:** Data should have shape (observations, frequencies, channels/vertices)

In [ ]:
epochs_power = list()
for condition in [epochs[k] for k in ('cCon', 'cInc')]:
    this_tfr = tfr_morlet(condition, freqs, n_cycles=n_cycles, average=False, return_itc=False)
    this_tfr.apply_baseline(mode='ratio', baseline=baseline)
    epochs_power.append(this_tfr.data)

# transpose again to (epochs, times, frequencies, vertices)
X = [np.transpose(x, (0, 3, 2, 1)) for x in epochs_power]

Adjacency matrix!

In [ ]:
adjacency, ch_names = find_ch_adjacency(epochs.info, ch_type='eeg')

print(type(adjacency))  # it's a sparse matrix!

fig, ax = plt.subplots(figsize=(5, 4))
ax.imshow(adjacency.toarray(), cmap='gray', origin='lower',
          interpolation='nearest')
ax.set_xlabel('{} Magnetometers'.format(len(ch_names)))
ax.set_ylabel('{} Magnetometers'.format(len(ch_names)))
ax.set_title('Between-sensor adjacency')
fig.tight_layout()

tfr_adjacency = combine_adjacency(
    len(freqs), len(this_tfr.times), adjacency)

In [ ]:
F_obs, clusters, p_values, _ = spatio_temporal_cluster_test(
    X, n_permutations=1000, tail=1, n_jobs=1, adjacency=tfr_adjacency)